### Which Python Packages are Included?

In [1]:
import requests
from requests.exceptions import RequestException
import random
import os
import logging
import re
import ast
import time
import csv
import json
import tempfile
import subprocess
from datetime import datetime
from tqdm import tqdm
from datetime import datetime
from typing import Set, List, Tuple, Dict, Any, Optional

In [2]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("repo_analysis.log"),
        logging.StreamHandler()
    ]
)

In [3]:
# Constants
GITHUB_API_URL = "https://api.github.com"
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"
]
GITHUB_TOKEN = ""
#os.environ.get("GITHUB_TOKEN")
DATA_FILE = "library_usage.csv"

### Utils

In [4]:
# Global variables to track rate limit status
remaining_requests = 3000  # Authenticated GitHub API limit (increased from 60)
rate_limit_reset_time = 0
last_rate_check = 0
RATE_LIMIT_WAIT = True  # Set to False to exit instead of waiting for rate limit reset
RETRY_DELAY = 1  # Reduced from 2

def get_headers() -> Dict[str, str]:
    """Get headers for API requests with random user agent and authentication if available."""
    headers = {
        "User-Agent": random.choice(USER_AGENTS),
        "Accept": "application/vnd.github.v3+json"
    }
    if GITHUB_TOKEN:
        headers["Authorization"] = f"token {GITHUB_TOKEN}"
    return headers

class RateLimitExceeded(Exception):
    """Exception raised when GitHub API rate limit is exceeded."""
    pass

def check_rate_limit() -> Tuple[int, int]:
    """
    Check GitHub API rate limit status.
    
    Returns:
        Tuple of (remaining requests, reset time in seconds)
    """
    global remaining_requests, rate_limit_reset_time, last_rate_check
    
    # Only check every 30 seconds to avoid wasting requests (increased from 10)
    current_time = time.time()
    if current_time - last_rate_check < 30:
        return remaining_requests, rate_limit_reset_time
    
    try:
        response = requests.get(f"{GITHUB_API_URL}/rate_limit", headers=get_headers())
        data = response.json()
        
        if 'resources' in data and 'core' in data['resources']:
            remaining_requests = data['resources']['core']['remaining']
            rate_limit_reset_time = data['resources']['core']['reset']
            last_rate_check = current_time
            
            logging.info(f"Rate limit status: {remaining_requests} requests remaining, " +
                        f"resets in {rate_limit_reset_time - current_time:.1f} seconds")
            
            return remaining_requests, rate_limit_reset_time
    except Exception as e:
        logging.warning(f"Failed to check rate limit: {e}")
    
    return remaining_requests, rate_limit_reset_time

def wait_for_rate_limit() -> bool:
    """
    Wait for rate limit to reset if needed.
    
    Returns:
        True if we waited and rate limit should be reset, False if program should exit
    """
    remaining, reset_time = check_rate_limit()
    
    # More aggressive approach - only wait when we have very few requests left
    if remaining > 1:  # Reduced buffer from 5 to 1
        return True
    
    wait_time = max(0, reset_time - time.time())
    
    if wait_time > 300 and not RATE_LIMIT_WAIT:  # More than 5 minutes wait
        logging.error(f"Rate limit exceeded. Would need to wait {wait_time/60:.1f} minutes. Exiting.")
        return False
    
    logging.warning(f"Rate limit almost reached. Waiting {wait_time:.1f} seconds for reset...")
    
    # Print countdown every 60 seconds (increased from 30)
    while wait_time > 0:
        time.sleep(min(60, wait_time))
        wait_time = max(0, reset_time - time.time())
        if wait_time > 0:
            logging.info(f"Still waiting for rate limit reset: {wait_time:.1f} seconds remaining")
    
    # Reset our tracking variables
    global remaining_requests, last_rate_check
    remaining_requests = 5000  # Default authenticated GitHub API limit (increased from 60)
    last_rate_check = 0
    
    logging.info("Rate limit should be reset now. Resuming operations.")
    return True

def make_github_request(url: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """
    Make a request to GitHub API with rate limit handling but no retries.
    
    Args:
        url: The API URL to request
        params: Optional query parameters
        
    Returns:
        Parsed JSON response
        
    Raises:
        RateLimitExceeded: If rate limit is hit and waiting is not possible
        RequestException: For other request errors
    """
    global remaining_requests, rate_limit_reset_time
    
    # Only check rate limit if we're getting close to the limit
    if remaining_requests <= 100:  # Only check when getting low
        if not wait_for_rate_limit():
            raise RateLimitExceeded("Rate limit exceeded and waiting is disabled")
    
    try:
        response = requests.get(url, headers=get_headers(), params=params)
        
        # Update rate limit info from response headers
        if 'X-RateLimit-Remaining' in response.headers:
            remaining_requests = int(response.headers['X-RateLimit-Remaining'])
            rate_limit_reset_time = int(response.headers['X-RateLimit-Reset'])
        
        # Handle rate limiting - no retry, just report the error
        if response.status_code == 403 and remaining_requests == 0:
            logging.error("Rate limit exceeded")
            raise RateLimitExceeded("Rate limit exceeded")
        
        # Special handling for common error codes
        if response.status_code == 404:
            logging.warning(f"Resource not found: {url}")
        elif response.status_code == 451:
            logging.warning(f"Resource unavailable for legal reasons (451): {url}")
        
        response.raise_for_status()
        return response.json()
    
    except requests.exceptions.HTTPError as e:
        logging.warning(f"HTTP error for {url}: {e}")
        raise
    except json.JSONDecodeError as e:
        logging.warning(f"JSON decode error for {url}: {e}")
        raise
    except RequestException as e:
        logging.warning(f"Request error for {url}: {e}")
        raise

### Get Random Python Repos

In [5]:
def get_random_python_repos(n: int = 10, min_stars: int = 0, max_per_page: int = None) -> List[Tuple[str, str, str]]:
    """
    Get random Python repositories from GitHub using the repositories endpoint with random IDs.
    Uses the utility functions for rate limiting and request handling.
    
    Args:
        n: Number of repositories to fetch
        min_stars: Minimum number of stars for repositories
        max_per_page: Maximum number of repositories to consider from each page (None = all)
        
    Returns:
        List of tuples (repo_name, repo_url, last_updated)
    """
    results = []
    
    while len(results) < n:
        try:
            # Generate a random repository ID in a reasonable range
            random_id = random.randint(1, 700_000_000)
            
            # Fetch repositories starting from random ID
            url = f"{GITHUB_API_URL}/repositories?since={random_id}"
            
            try:
                response = make_github_request(url)
                repos = response
            except RateLimitExceeded:
                logging.warning("Rate limit exceeded while fetching random repositories. Waiting...")
                if not wait_for_rate_limit():
                    logging.error("Could not wait for rate limit reset. Exiting.")
                    break
                continue
            
            # Limit number of repos to consider from this page if specified
            if max_per_page:
                repos = repos[:max_per_page]
            
            # Process each repository
            for repo in repos:
                # Get repository name
                repo_name = repo.get("full_name")
                if not repo_name:
                    continue
                
                try:
                    # Check if it's a Python repository and meets star criteria
                    repo_url = f"{GITHUB_API_URL}/repos/{repo_name}"
                    repo_data = make_github_request(repo_url)
                    
                    # Check language and stars
                    if (repo_data.get("language") == "Python" and 
                            repo_data.get("stargazers_count", 0) >= min_stars):
                        
                        results.append((
                            repo_name,
                            repo_data.get("html_url", ""),
                            repo_data.get("updated_at", "")
                        ))
                        logging.info(f"Found: {repo_name} (Stars: {repo_data.get('stargazers_count')})")
                        
                        # Check if we have enough results
                        if len(results) >= n:
                            break
                
                except RateLimitExceeded:
                    logging.warning("Rate limit exceeded while fetching repo details. Waiting...")
                    if not wait_for_rate_limit():
                        logging.error("Could not wait for rate limit reset. Exiting.")
                        break
                except Exception as e:
                    logging.warning(f"Error processing repository {repo_name}: {e}")
                    continue
                
        except Exception as e:
            logging.error(f"Error in repository fetch: {e}")
            # Exponential backoff on errors
            time.sleep(10)
    
    logging.info(f"Successfully collected {len(results)} random Python repositories")
    return results[:n]

### Get Python Files and Analyze Files per Repo.

In [6]:
def get_default_branch(repo_name: str) -> Optional[str]:
    """
    Get the default branch (main or master) for a repository.
    Returns None if the branch cannot be determined.
    """
    try:
        if not wait_for_rate_limit():
            raise RateLimitExceeded("Rate limit exceeded while getting default branch")
            
        response = make_github_request(f"{GITHUB_API_URL}/repos/{repo_name}")
        default_branch = response.get("default_branch", "main")
        return default_branch
    except Exception as e:
        logging.warning(f"Couldn't determine default branch for {repo_name}: {e}")
        # Return None to indicate we couldn't get the branch
        return None

def get_python_files(repo_name: str, max_files: int = 10, seed: int = 42) -> List[str]:
    """
    Get a list of Python files in the repository.
    
    Args:
        repo_name: Full name of the repository (owner/repo)
        max_files: Maximum number of files to return
        seed: Random seed for reproducible file selection
        
    Returns:
        List of file paths
    """
    try:
        if not wait_for_rate_limit():
            raise RateLimitExceeded("Rate limit exceeded while getting Python files")
        
        default_branch = get_default_branch(repo_name)
        if default_branch is None:
            logging.warning(f"Skipping repo {repo_name} - couldn't determine default branch")
            return []
            
        url = f"{GITHUB_API_URL}/repos/{repo_name}/git/trees/{default_branch}?recursive=1"
        
        try:
            response = make_github_request(url)
            
            if 'tree' in response:
                python_files = [file['path'] for file in response['tree'] 
                        if file['path'].endswith('.py') and file['type'] == 'blob']
                
                # Set seed and shuffle to ensure reproducible file selection
                random.seed(seed)
                random.shuffle(python_files)
                python_files = python_files[:max_files]
                
                return python_files
        except RateLimitExceeded:
            logging.warning(f"Rate limit exceeded while getting files for {repo_name}")
            # Try fallback method
        
        # Fallback method: try to get files from the repo contents
        url = f"{GITHUB_API_URL}/repos/{repo_name}/contents"
        response = make_github_request(url)
        
        python_files = []
        for item in response:
            if item['type'] == 'file' and item['name'].endswith('.py'):
                python_files.append(item['path'])
            elif item['type'] == 'dir':
                # Try to look in top-level directories
                try:
                    dir_url = f"{GITHUB_API_URL}/repos/{repo_name}/contents/{item['path']}"
                    dir_response = make_github_request(dir_url)
                    
                    for dir_item in dir_response:
                        if dir_item['type'] == 'file' and dir_item['name'].endswith('.py'):
                            python_files.append(dir_item['path'])
                except Exception:
                    pass
                    
        # Set seed and shuffle files from fallback method as well
        random.seed(seed)
        random.shuffle(python_files)
        python_files = python_files[:max_files]
        
        return python_files
                
    except Exception as e:
        logging.error(f"Error getting Python files for {repo_name}: {e}")
        return []

def extract_imports(file_content: str) -> Set[str]:
    """
    Extract all imported module names from Python code using the ast module.
    
    Args:
        file_content: Python code content as a string
        
    Returns:
        Set of top-level module names that are imported
    """
    try:
        # Parse the code into an AST
        tree = ast.parse(file_content)
        
        imports = set()
        
        # Walk through the AST nodes
        for node in ast.walk(tree):
            # Handle "import module" or "import module.submodule" statements
            if isinstance(node, ast.Import):
                for name in node.names:
                    # Extract the top-level module name (before any dots)
                    module_name = name.name.split('.', 1)[0]
                    imports.add(module_name)
            
            # Handle "from module import name" statements
            elif isinstance(node, ast.ImportFrom):
                if node.module:  # In case of "from . import name"
                    # Extract the top-level module name (before any dots)
                    module_name = node.module.split('.', 1)[0]
                    imports.add(module_name)
        
        return imports
    
    except SyntaxError:
        # Fallback to regex for files with syntax errors
        fallback_imports = set()
        lines = file_content.split("\n")
        for line in lines:
            # Match import statements
            import_match = re.match(r'\s*import\s+([\w\s,]+)(?:\s+as\s+\w+)?', line)
            if import_match:
                modules = import_match.group(1).split(',')
                for module in modules:
                    module_name = module.strip().split('.', 1)[0].split()[0]
                    fallback_imports.add(module_name)
                    
            # Match from statements
            from_match = re.match(r'\s*from\s+(\w+)', line)
            if from_match:
                fallback_imports.add(from_match.group(1))
                
        return fallback_imports

def extract_imports(content: str) -> Set[str]:
    """
    Extract imported libraries from Python content using AST parsing
    and regex as a fallback for edge cases.
    """
    libraries = set()
    
    # AST parsing for reliable import extraction
    try:
        tree = ast.parse(content)
        for node in ast.walk(tree):
            if isinstance(node, ast.Import):
                for name in node.names:
                    libraries.add(name.name.split('.')[0])
            elif isinstance(node, ast.ImportFrom):
                if node.module:
                    libraries.add(node.module.split('.')[0])
    except SyntaxError:
        # Fallback to regex for files with syntax errors
        import_pattern = r'^import\s+([\w\.]+)|^from\s+([\w\.]+)\s+import'
        for match in re.finditer(import_pattern, content, re.MULTILINE):
            lib = match.group(1) or match.group(2)
            if lib:
                libraries.add(lib.split('.')[0])
                
    return libraries

def analyze_repo(repo_info: Tuple[str, str, str]) -> List[Tuple[str, str, str, str, str]]:
    """
    Analyze a GitHub repository for Python library usage by cloning it once.
    
    Args:
        repo_info: Tuple of (repo_name, repo_url, last_updated)
        
    Returns:
        List of tuples (library_name, repo_name, file_path, fetch_date, last_updated)
    """
    repo_name, repo_url, last_updated = repo_info
    logging.info(f"Processing repo: {repo_name}")
    
    repo_results = []
    fetch_date = datetime.utcnow().isoformat()
    
    # Create a temporary directory for the cloned repo
    with tempfile.TemporaryDirectory() as temp_dir:
        try:
            # Clone the repository with minimal depth
            clone_cmd = f"git clone --depth 1 https://github.com/{repo_name}.git {temp_dir}"
            subprocess.run(clone_cmd, shell=True, check=True, capture_output=True)
            
            # Find Python files in the repository
            python_files = []
            for root, _, files in os.walk(temp_dir):
                for file in files:
                    if file.endswith('.py'):
                        relative_path = os.path.relpath(os.path.join(root, file), temp_dir)
                        python_files.append(relative_path)
                
                # Limit the number of files to analyze if there are too many
                if len(python_files) >= 10:
                    python_files = python_files[:10]
                    break
            
            # Process each Python file
            for file_path in python_files:
                try:
                    full_path = os.path.join(temp_dir, file_path)
                    with open(full_path, 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()
                    
                    imports = extract_imports(content)
                    
                    for library in imports:
                        repo_results.append((
                            library, 
                            repo_name, 
                            file_path, 
                            fetch_date, 
                            last_updated
                        ))
                except Exception as e:
                    logging.warning(f"Error processing file {file_path} in {repo_name}: {e}")
            
            return repo_results
        
        except Exception as e:
            logging.error(f"Failed to analyze repo {repo_name}: {e}")
            return []

def save_results(results: List[Tuple[str, str, str, str, str, bool]]) -> None:
    """Save analysis results to CSV file with proper error handling."""
    if not results:
        logging.warning("No results to save")
        return
    
    try:
        file_exists = os.path.exists(DATA_FILE)
        
        with open(DATA_FILE, "a", newline='') as f:
            writer = csv.writer(f)
            if not file_exists:
                writer.writerow([
                    "library_name", 
                    "repo", 
                    "file", 
                    "fetch_date", 
                    "last_updated" 
                ])
            writer.writerows(results)
        
        logging.info(f"Saved {len(results)} entries to {DATA_FILE}")
    
    except Exception as e:
        logging.error(f"Error saving results to CSV: {e}")
        
        # Emergency backup
        backup_file = f"backup_results_{int(time.time())}.json"
        with open(backup_file, "w") as f:
            json.dump(results, f)
        logging.info(f"Backup saved to {backup_file}")

In [7]:
def main():
    """Main function to orchestrate the GitHub repository analysis."""
    try:
        num_repos = 50  # Adjust as needed
        logging.info(f"Fetching {num_repos} repositories...")
        sampled_repos = get_random_python_repos(n=num_repos, min_stars=0)
        logging.info(f"Fetched {len(sampled_repos)} repositories for analysis")
        
        # Create/initialize the CSV file before processing
        file_exists = os.path.exists(DATA_FILE)
        if not file_exists:
            with open(DATA_FILE, "w", newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "library_name", 
                    "repo", 
                    "file", 
                    "fetch_date", 
                    "last_updated" 
                ])
            logging.info(f"Created new CSV file: {DATA_FILE}")
        
        total_imports = 0
        
        # Process repositories sequentially with progress bar
        for repo in tqdm(sampled_repos, desc="Processing Repositories", unit="repo"):
            try:
                repo_name = repo[0]
                tqdm.write(f"Processing {repo_name}...")
                
                repo_results = analyze_repo(repo)
                
                # Write results to CSV immediately after each repo
                if repo_results:
                    with open(DATA_FILE, "a", newline='') as f:
                        writer = csv.writer(f)
                        writer.writerows(repo_results)
                    
                    total_imports += len(repo_results)
                    tqdm.write(f"Completed {repo_name}: saved {len(repo_results)} library imports (total: {total_imports})")
                else:
                    tqdm.write(f"Completed {repo_name}: no library imports found")
                    
            except Exception as e:
                tqdm.write(f"Error processing {repo[0]}: {e}")
                error_file = f"error_{repo[0].replace('/', '_')}_{int(time.time())}.txt"
                with open(error_file, "w") as f:
                    f.write(f"Error processing {repo[0]}: {str(e)}\n")
                    f.write(traceback.format_exc())
        
        logging.info(f"Analysis complete. Analyzed {total_imports} library imports across {len(sampled_repos)} repositories")
        logging.info(f"Results saved to {DATA_FILE}")
        
    except Exception as e:
        logging.error(f"Fatal error in main process: {str(e)}")
        traceback.print_exc()
        sys.exit(1)
        
    return 0

if __name__ == "__main__":
    exit_code = main()
    sys.exit(exit_code)

2025-03-12 20:12:35,226 - INFO - Fetching 50 repositories...
2025-03-12 20:12:37,380 - WARNING - Resource not found: https://api.github.com/repos/awsassets/ipfs-crawler
2025-03-12 20:12:37,382 - WARNING - HTTP error for https://api.github.com/repos/awsassets/ipfs-crawler: 404 Client Error: Not Found for url: https://api.github.com/repos/awsassets/ipfs-crawler
2025-03-12 20:12:37,383 - WARNING - Error processing repository awsassets/ipfs-crawler: 404 Client Error: Not Found for url: https://api.github.com/repos/awsassets/ipfs-crawler
2025-03-12 20:12:39,031 - INFO - Found: shafiurrahman/heroku-salary-prediction (Stars: 0)
2025-03-12 20:12:39,470 - WARNING - HTTP error for https://api.github.com/repos/Gomes09/MagiskOnWSA: 403 Client Error: Forbidden for url: https://api.github.com/repos/Gomes09/MagiskOnWSA
2025-03-12 20:12:39,470 - WARNING - Error processing repository Gomes09/MagiskOnWSA: 403 Client Error: Forbidden for url: https://api.github.com/repos/Gomes09/MagiskOnWSA
2025-03-12 20

Processing shafiurrahman/heroku-salary-prediction...


Processing Repositories:   2%|▎                | 1/50 [00:01<01:26,  1.76s/repo]2025-03-12 20:16:40,754 - INFO - Processing repo: David-coder02/AirBnB_clone


Completed shafiurrahman/heroku-salary-prediction: saved 7 library imports (total: 7)
Processing David-coder02/AirBnB_clone...


Processing Repositories:   4%|▋                | 2/50 [00:02<01:04,  1.34s/repo]2025-03-12 20:16:41,762 - INFO - Processing repo: matt5797/rawsocket_sniffer


...

Processing Repositories:  98%|███████████████▋| 49/50 [00:53<00:00,  1.32repo/s]2025-03-12 20:17:32,750 - INFO - Processing repo: leafiy/sublime-less2css


Completed MeGaPk/makerbot-gen5-api: saved 43 library imports (total: 808)
Processing leafiy/sublime-less2css...


Processing Repositories: 100%|████████████████| 50/50 [00:54<00:00,  1.09s/repo]
2025-03-12 20:17:33,544 - INFO - Analysis complete. Analyzed 824 library imports across 50 repositories
2025-03-12 20:17:33,544 - INFO - Results saved to library_usage.csv


Completed leafiy/sublime-less2css: saved 16 library imports (total: 824)


NameError: name 'sys' is not defined